In [1]:
!pip install nibabel monai timm torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.0 MB/s eta 0:00:00:00:0100:01
  Attempting unins

In [16]:
#clear directory
!cd /kaggle/working/
!rm -rf *

In [2]:
!pip install gdown


In [17]:
import gdown

#https://drive.google.com/file/d/1tUArfL0QHhd0XkfXX1FnExF8_PbABtDQ/view?usp=sharing

file_id = "1tUArfL0QHhd0XkfXX1FnExF8_PbABtDQ"  # replace with your file ID
url = f"https://drive.google.com/uc?id={file_id}"

output_path = "/kaggle/working/kits23.zip"
gdown.download(url, output_path, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1tUArfL0QHhd0XkfXX1FnExF8_PbABtDQ
From (redirected): https://drive.google.com/uc?id=1tUArfL0QHhd0XkfXX1FnExF8_PbABtDQ&confirm=t&uuid=1508cf60-76c8-4bcb-9189-493ef84158f3
To: /kaggle/working/kits23.zip
100%|██████████| 1.28G/1.28G [00:05<00:00, 240MB/s]


'/kaggle/working/kits23.zip'

In [18]:
import zipfile

zip_path = "/kaggle/working/kits23.zip"
extract_path = "/kaggle/working/kits23"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [5]:
import os, torch, nibabel as nib
import numpy as np
from torch.utils.data import Dataset, DataLoader

class KiTS23_2p5D(Dataset):
    def __init__(self, root_dir, cases, num_slices=3, transform=None):
        self.root_dir = root_dir
        self.cases = cases
        self.num_slices = num_slices
        self.transform = transform
        self.samples = []
        self._prepare_samples()

    def _prepare_samples(self):
        for case in self.cases:
            img_path = os.path.join(self.root_dir, case, "imaging.nii.gz")
            lbl_path = os.path.join(self.root_dir, case, "segmentation.nii.gz")
            img = nib.load(img_path).get_fdata()
            lbl = nib.load(lbl_path).get_fdata()
            img = (img - np.mean(img)) / (np.std(img) + 1e-5)

            for z in range(1, img.shape[2]-1):
                self.samples.append((img[..., z-1:z+2], lbl[..., z]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        stack, mask = self.samples[idx]
        stack = torch.tensor(stack).float().permute(2, 0, 1)  # (3,H,W)
        mask = torch.tensor(mask).long()  # (H,W)
        return stack, mask


In [6]:
root = "/kaggle/working/kits23/kits23"
cases = sorted(os.listdir(root))[:10]  # use only 50 cases

train_cases = cases[:7]
val_cases = cases[7:]

train_ds = KiTS23_2p5D(root, train_cases)
val_ds   = KiTS23_2p5D(root, val_cases)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=4)


In [7]:
import torch
print(torch.cuda.is_available())  # should be True
print(torch.cuda.device_count())  # should match Kaggle GPU (1 or 2)
print(torch.cuda.get_device_name(0))


True
2
Tesla T4


In [10]:
import torch.nn as nn
from monai.networks.blocks import UnetOutBlock
from timm.models.vision_transformer import vit_base_patch16_224

class TransUNet2p5D(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()
        self.vit = vit_base_patch16_224(pretrained=True)
        self.vit.conv_proj = nn.Conv2d(in_channels, 768, kernel_size=16, stride=16)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(768, 256, 2, 2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 2, 2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 2, 2),
            nn.ReLU(),
            nn.Conv2d(64, out_channels, 1)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        y = self.vit.conv_proj(x)
        y = y.flatten(2).transpose(1, 2)
        y = self.vit.blocks(y)
        y = y.transpose(1, 2).reshape(B, 768, H//16, W//16)
        y = self.decoder(y)
        return y


In [11]:
import torch.nn.functional as F

def dice_loss(pred, target, eps=1e-5):
    pred = torch.softmax(pred, dim=1)
    target_1hot = F.one_hot(target, num_classes=3).permute(0,3,1,2)
    inter = (pred * target_1hot).sum(dim=(2,3))
    denom = (pred + target_1hot).sum(dim=(2,3))
    dice = (2 * inter / (denom + eps)).mean()
    return 1 - dice

model = TransUNet2p5D(in_channels=3, out_channels=3).cuda()
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [12]:
for epoch in range(10):  # 10–20 epochs enough for baseline
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()
        out = model(x)
        loss = dice_loss(out, y)
        opt.zero_grad(); loss.backward(); opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}: Train DiceLoss {total_loss/len(train_loader):.4f}")


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/collate.py", line 211, in collate
    return [
           ^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/collate.py", line 212, in <listcomp>
    collate(samples, collate

new

In [14]:
import os
import nibabel as nib
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class KiTS23_2p5D_Resized(Dataset):
    def __init__(self, root_dir, cases, num_slices=3, target_size=(256, 256)):
        """
        root_dir: folder containing case_00000, case_00001, ...
        cases: list of case folder names to use
        num_slices: number of consecutive slices for 2.5D input (usually 3)
        target_size: (H, W) to resize each slice
        """
        self.root_dir = root_dir
        self.cases = cases
        self.num_slices = num_slices
        self.target_size = target_size
        self.samples = []
        self._prepare_samples()

    def _prepare_samples(self):
        for case in self.cases:
            img_path = os.path.join(self.root_dir, case, "imaging.nii.gz")
            lbl_path = os.path.join(self.root_dir, case, "segmentation.nii.gz")

            img = nib.load(img_path).get_fdata()
            lbl = nib.load(lbl_path).get_fdata()

            # Normalize intensity
            img = (img - np.mean(img)) / (np.std(img) + 1e-5)

            # Generate 2.5D slices
            for z in range(1, img.shape[2]-1):
                self.samples.append((img[..., z-1:z+2], lbl[..., z]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        stack, mask = self.samples[idx]

        # Convert to torch tensors
        stack = torch.tensor(stack).float().permute(2, 0, 1)  # (C,H,W)
        mask = torch.tensor(mask).long()                       # (H,W)

        # Resize stack and mask
        stack = F.interpolate(stack.unsqueeze(0), size=self.target_size, mode='bilinear', align_corners=False).squeeze(0)
        mask = F.interpolate(mask.unsqueeze(0).unsqueeze(0).float(), size=self.target_size, mode='nearest').squeeze(0).long()

        return stack, mask


In [19]:
root = "/kaggle/working/kits23/kits23"
cases = sorted(os.listdir(root))[:10]  # use only 50 cases
train_cases = cases[:7]
val_cases = cases[7:]

train_ds = KiTS23_2p5D_Resized(root, train_cases)
val_ds   = KiTS23_2p5D_Resized(root, val_cases)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=4, num_workers=2)
